In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Aim is to predict the marks of students of the test data

def feature_changing(x_data):
    internet_encoded = np.array([1 if x == 'yes' else 0 for x in x_data[:, 0]]).reshape(-1, 1)
    other_features = x_data[:, 1:] 
    x_data_new = np.hstack((internet_encoded, other_features))
    return x_data_new

def z_score(x_train):
    x_mean = np.mean(x_train, axis=0)
    x_std = np.std(x_train, axis=0)
    x_std[x_std == 0] = 1e-10
    x_train_scaled = (x_train - x_mean) / x_std
    return x_train_scaled, x_std, x_mean

# Define cost function
def cost(x_train, y_train, w, b):
    m = x_train.shape[0]
    predictions = np.dot(x_train, w) + b
    loss = np.sum((predictions - y_train)**2) / (2 * m)
    return loss

# Define gradient_descent function
def gradient_descent(x_train, y_train, w, b, learning_rate=0.01, iterations=1000):
    m = x_train.shape[0]
    for _ in range(iterations):
        predictions = np.dot(x_train, w) + b
        errors = predictions - y_train
        
        dj_dw = np.dot(x_train.T, errors) / m
        dj_db = np.sum(errors) / m
        
        w = w - learning_rate * dj_dw
        b = b - learning_rate * dj_db
    return w, b

# Load training data
data = pd.read_excel('Trainingdata.xlsx')
x_train = np.array(data.iloc[:,0:8])
y_train = np.array(data.iloc[:,8]).reshape(-1,1)

x_train = feature_changing(x_train)
x_train = x_train.astype(np.float64)
x_train,x_std,x_mean = z_score(x_train)

np.random.seed(2147483647)
w = np.random.randn(x_train.shape[1],1)
b = np.random.randn(1)

old_cost = 0

while abs(old_cost - cost(x_train,y_train,w,b))>0.00001:
  old_cost = cost(x_train,y_train,w,b)
  w,b = gradient_descent(x_train,y_train,w,b)

x_predict = pd.read_excel('Test data.xlsx').iloc[:,:8].to_numpy()
x_predict = feature_changing(x_predict)
x_predict = (x_predict - x_mean)/x_std
ans = pd.read_excel('Test data.xlsx').iloc[:,8].to_numpy()

y_predict = np.dot(x_predict,w) + b

accuracy = 0
for dim in range(len(ans)):
  if abs(y_predict[dim]-ans[dim])<0.5: 
    accuracy += 1
accuracy = round(accuracy*100/200.0,2)
ok = 'Congratulations' if accuracy>95 else 'Optimization required'
print(f"{ok}, your accuracy is {accuracy}%")
